In [1]:
### get current timestamp
import datetime
from pytz import timezone

curent_timestamp = datetime.datetime.now().astimezone(timezone('US/Pacific')).strftime("%Y_%m_%d_%H_%M_%S")

### Create horovod_benchmark folder under mounted s3 bucket
import os
final_output_dir = "/dbfs/mnt/wendao_test2/horovod_benchmark/logs/"
output_dir = 'logs/logs/horovod_logs/'

output_dir = output_dir + f"{curent_timestamp}/"
model_output_dir = output_dir + "model/"

for _path in [output_dir, final_output_dir, model_output_dir]:
  try:
    os.makedirs(_path)
    print(f"{_path} - file created successfully")
  except Exception as ex:
    if FileExistsError:
      print(f"{_path} - file already exist")
    else:
      raise ex

logs/logs/horovod_logs/2019_11_11_01_24_08/ - file created successfully
/dbfs/mnt/wendao_test2/horovod_benchmark/logs/ - file already exist
logs/logs/horovod_logs/2019_11_11_01_24_08/model/ - file created successfully

In [2]:
import logging
import sys
import datetime
import glob


### TO DO 
### ADDING SINGLE_INSTANCE, REPEAT INTO LOGGER STRING FORMAT
original_stdout = sys.stdout
original_stderr = sys.stderr

def redirect_stdout(log_filename):
  ### Databricks has weird behavior that need to define redirect stdout in every cell. -- Need follow up with databricks
  ### https://stackoverflow.com/questions/34248908/how-to-prevent-logging-of-pyspark-answer-received-and-command-to-send-messag
  ### logging flushed with py4j error messages so to set logging level to ERROR to silence error
  ### Also leave pyspark/matplotlib silence code just in case
  logging.getLogger("py4j").setLevel(logging.ERROR)
  ##logging.getLogger('pyspark').setLevel(logging.ERROR)
  ##matplotlib_logger = logging.getLogger("matplotlib").setLevel(logging.ERROR)
  
  class StreamToLogger:
    def __init__(self, logger, level):
        # self.level is really like using log.debug(message)
        # at least in my case
        self.level = level
        self.logger = logger

    def write(self, message):
        # if statement reduces the amount of newlines that are
        # printed to the logger
        if (message != '\n') & (message != ''):
          for mess in message.splitlines():
            self.logger.log(self.level, mess.rstrip())
            
    def flush(self):
      pass
#     def flush(self):
#         # create a flush method so things can be flushed when
#         # the system wants to. Not sure if simply 'printing'
#         # sys.stderr is the correct way to do it, but it seemed
#         # to work properly for me.
#         self.level(sys.stderr)
        
  ## Change to pdt timezone (Simply implementation won't consider daylight savings)
  def pdt_timezone(sec, what):
    pdt_time = datetime.datetime.now() - datetime.timedelta(hours=8)
    return pdt_time.timetuple()
  logging.Formatter.converter = pdt_timezone
  
    
  logging.basicConfig(
     level=logging.DEBUG,
     ## format='%(asctime)-%(levelname)s-%(name)s-%(message)s',
     format=('%(asctime)s - %(name)s - %(levelname)s - %(message)s'),
     datefmt="%Y-%m-%d %H:%M:%S",
     filename = log_filename,
     filemode = 'w'
  )
         
  stdout_logger = logging.getLogger('STDOUT')
  sl = StreamToLogger(stdout_logger, logging.INFO)
  sys.stdout = sl
  
  stderr_logger = logging.getLogger('STDERR')
  sle = StreamToLogger(stderr_logger, logging.ERROR)
  sys.stderr = sle
  
def reset_stdout():
  sys.stdout = original_stdout
  sys.stderr = original_stderr
  temp_logger = logging.getLogger()
  temp_logger.handlers = []
  
  
def move_log_to_s3():
  ### Moving all the driver logs to mounted s3 bucket. 
  import glob
  from distutils import dir_util 
  ### Shutile will not overwrit the file so use distutils instead

  list_dir = glob.glob(output_dir)

  for path in list_dir:
    dir_name = os.path.basename(os.path.dirname(path))
    dest = final_output_dir + dir_name
    destination = dir_util.copy_tree(path, dest) 
    
def save_horovod_model():
  import shutil
  for path in glob.glob(checkpoint_dir+"/*"):
    desc_dir = model_output_dir + "NP" +str(np_setup) + "/"
    if not os.path.exists(desc_dir):
      os.makedirs(desc_dir)
    model_filename = os.path.basename(path)
    shutil.copy(path, desc_dir + model_filename)


In [3]:
dbutils.widgets.removeAll()
dbutils.widgets.text(name = "batch_size", defaultValue = "128")
dbutils.widgets.text(name = "learning_rate", defaultValue = "0.1")
dbutils.widgets.text(name = "epochs", defaultValue = "5")
dbutils.widgets.text(name = "repeat", defaultValue = "3")
dbutils.widgets.text(name = "np", defaultValue = "8")

In [4]:
batch_size = int(dbutils.widgets.get("batch_size"))
epochs = int(dbutils.widgets.get("epochs"))
repeat = int(dbutils.widgets.get("repeat"))
learning_rate = float(dbutils.widgets.get("learning_rate"))
np_setup = float(dbutils.widgets.get("np"))

In [5]:
import time
import os
import datetime
import time
import pandas as pd 
import numpy as np


In [6]:
### Removed BACKGROUND_Google folder 
### Why there are two faces folder - Faces and Faces_easy

In [7]:
HEIGHT=224 #VGG16 224 VS InceptionV3 299
WIDTH=224 #VGG16 224 VS InceptionV3 299
TRAIN_DIR="/dbfs/tmp/caltech/train/"  #need to add train
TEST_DIR="/dbfs/tmp/caltech/test/"   #need to add test
NUM_CLASSES = 101 # number of category

In [8]:
train_files = os.listdir(TRAIN_DIR)
total_train_files = 0
train_class = 0
for i in train_files:
  i = TRAIN_DIR + '/' + i
  if os.path.isdir(i):
    total_train_files += len(os.listdir(i))
    train_class += 1
  
test_files = os.listdir(TEST_DIR)
total_test_files = 0
test_class = 0
for i in test_files:
  i = TEST_DIR + '/' + i
  if os.path.isdir(i):
    total_test_files += len(os.listdir(i))
    test_class += 1

print(f"Train - {train_class} classes contain {total_train_files} pictures")
print(f"Test - {test_class} classes contain {total_test_files} pictures")

Train - 101 classes contain 6982 pictures
Test - 101 classes contain 1695 pictures

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import time
# https://github.com/CyberZHG/keras-radam/issues/12
# os.environ['TF_KERAS'] = '1' #apparently it doens't work, but %env tf_keras=1 works. everytime tried after clear state of the notebook

In [10]:
## %env TF_KERAS =1
## https://github.com/CyberZHG/keras-radam/issues/12
## https://github.com/CyberZHG/keras-lookahead/issues/1

In [11]:
#tf.keras
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import horovod.tensorflow.keras as hvd
from tensorflow.keras import backend as K
import tensorflow as tf
#np.set_printoptions(threshold=np.inf)

#https://pypi.org/project/keras-rectified-adam/
# from keras_radam import RAdam

/databricks/python/lib/python3.7/site-packages/tensorflow/python/util/nest.py:1286: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 _pywrap_tensorflow.RegisterType("Mapping", _collections.Mapping)
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/horovod/tensorflow/__init__.py:117: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/horovod/tensorflow/__init__.py:143: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

In [12]:
### Try to write generator into pickle or other format but doesn't work, so each nodes need process this step separately
def get_dataset():

  #https://stackoverflow.com/questions/44341258/preprocessing-function-of-inception-v3-in-keras
  datagen = ImageDataGenerator(
          rotation_range=180, # I really think unlike cats and dogs, pill image can be rotated 180 degrees
          preprocessing_function=vgg16_preprocess_input,
          width_shift_range=0.2,
          height_shift_range=0.2,
          #rescale=1./255, #note that prepocess_input will rescale, no need to do additional rescale. 
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=True,
          fill_mode='nearest')

  # Images will be directly taken form our defined folder structure using the method flow_from_directory()
  train_generator =datagen.flow_from_directory(
      TRAIN_DIR,
      target_size=(HEIGHT, WIDTH),
          batch_size=batch_size,
          class_mode='categorical')

  validation_generator = datagen.flow_from_directory(
      TEST_DIR,
      target_size=(HEIGHT, WIDTH),
      batch_size=batch_size,
      class_mode='categorical')
  
  return (train_generator,validation_generator)

In [13]:
#later need to change to get mdoel for multiple models.
#https://www.programcreek.com/python/example/106229/keras.applications.inception_v3.InceptionV3

def get_model(num_classes = NUM_CLASSES):
  base_model = VGG16(weights='imagenet',include_top=True) #top means the last 4 (flatten fc1, fc2, dense) layers. we only need to change dense layer. the rest are convolutional layers. 
  x = base_model.get_layer('fc2').output
  predictions = Dense(NUM_CLASSES, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  return model

In [14]:
test_model = get_model()

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/nn_ops.py:61: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 elif not isinstance(value, collections.Sized):
Downloading data from https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5
 8192/553467096 [..............................] - ETA: 1:25:15 40960/553467096 [..............................] - ETA: 34:06  90112/553467096 [..............................] - ETA: 23:21 139264/553467096 [..............................] - ETA: 20:05 319488/553467096 [..............................] - ETA: 10:56 630784/553467096 [..............................] - ETA: 6:38  1294336/553467096 [..............................] - ETA: 3:46 2613248/553467096 [..............................] - ETA: 2:07 4186112/553467096 [..............................] - ETA: 1:29 5758976/553467096 [..............................] - ETA: 1:12 7331840/553467096 [..............................] - ETA: 1:02 8904704/553467096 [..............................] - ETA: 55s  10477568/553467096 [..............................] - ETA: 51s 12050432/553467096 [..............................] - ETA: 47s 13623296/553467096 [..............................] - ETA: 45s 15196160/553467096 [..............................] - ETA: 42s 16769024/553467096 [..............................] - ETA: 41s 18341888/553467096 [..............................] - ETA: 39s 19914752/553467096 [>.............................] - ETA: 38s 21487616/553467096 [>.............................] - ETA: 37s 23060480/553467096 [>.............................] - ETA: 36s 24633344/553467096 [>.............................] - ETA: 35s 26206208/553467096 [>.............................] - ETA: 35s 27779072/553467096 [>.............................] - ETA: 34s 29351936/553467096 [>.............................] - ETA: 33s 30924800/553467096 [>..

In [15]:
test_model.save("logs/vgg_model.h5")

In [16]:
import shutil
shutil.copy("logs/vgg_model.h5", "/dbfs/mnt/wendao_test2/horovod_benchmark/preload_model/vgg_model.h5")

Out[16]: '/dbfs/mnt/wendao_test2/horovod_benchmark/preload_model/vgg_model.h5'

In [17]:
def train(learning_rate=learning_rate):
  ### Get dataset 
  train_generator, validation_generator = get_dataset()
  
  ### Get model
  model = get_model()

  ### Model Compile 
  # optimizer = RAdam(total_steps=5000, warmup_proportion=0.1, min_lr=1e-5)
  optimizer = keras.optimizers.Adadelta(lr=learning_rate)
  
  model.compile(optimizer = optimizer,#'rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  history = model.fit_generator(
      train_generator,
      epochs=epochs,
      validation_data=validation_generator,
      verbose=2
      #validation_steps=VALIDATION_STEPS,steps_per_epoch=STEPS_PER_EPOCH, 
  )
  ### TO DO SAVE THE MODEL
  # model.save("model_radam.h5")

In [18]:
### Define output log file name 
env = "PROD"
instance_type = 'SINGLE_INSTANCE'
dataset = 'CALTECH101'
model_name = 'VGG16'

train_shape = total_train_files
validation_shape = total_test_files

def get_cluster_info(hvd_run=True):
  driver_type = sc.getConf().get("spark.databricks.driverNodeTypeId")
  worker_type = "None"
  num_workers = "None"
  if hvd_run == True:
    worker_type = sc.getConf().get("spark.databricks.workerNodeTypeId")
    num_workers = sc.getConf().get("spark.databricks.clusterUsageTags.clusterWorkers")
  return [driver_type, worker_type, num_workers]

driver_type, worker_type, num_workers = get_cluster_info()
np_setup = "NA"

In [19]:
filename = f"benchmark|{env}|{dataset}|{model_name}|{train_shape}|{validation_shape}|{instance_type}|{driver_type}|{worker_type}|{num_workers}|{np_setup}|{repeat}|{epochs}|{learning_rate}|{batch_size}.log"

In [20]:
# reset_stdout()
# print(output_dir+filename)

# redirect_stdout(output_dir+filename)
# for i in range(repeat):
#   print(f"REPEAT {i+1}")
#   model = train()
# move_log_to_s3()
# reset_stdout()

# ### batch_size 128 -> OOM

# ### Lots of error closing message in logs: PIL.Image - DEBUG - Error closing: 'Image' object has no attribute 'fp'
# ### Seems like no harm but might need check it tomorrow
# ### https://github.com/romainbrette/holypipette/issues/80

In [22]:
#fit_generator version of hvd
#https://github.com/horovod/horovod/blob/master/examples/keras_mnist_advanced.py
#btw, the advanced example used fancy adadelta optimizer, that's why there are so many call backs.
#https://docs.databricks.com/applications/deep-learning/distributed-training/mnist-tensorflow-keras.html
#my understanding is that the entire script of horovod exampe needs to be put in train_hvd fundtion

def train_hvd(learning_rate = learning_rate):
  # Horovod: initialize Horovod.
  hvd.init()

  # Horovod: pin GPU to be used to process local rank (one GPU per process)
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.gpu_options.visible_device_list = str(hvd.local_rank())
  K.set_session(tf.Session(config=config))

  
  # Collect get data time
  start_time = time.time()
  
  train_generator, validation_generator = get_dataset()
  step_size_train = train_generator.n//train_generator.batch_size
  step_size_validation = validation_generator.n//validation_generator.batch_size
  
  end_time = round((time.time() - start_time),3)
  print(f"step - get_data - {end_time}")  

  
  # Model Compiles
  start_time = time.time()

  model=get_model()
#   model = keras.models.load_model("/dbfs/mnt/wendao_test2/horovod_benchmark/preload_model/vgg_model.h5")
  
  # Horovod: adjust learning rate based on number of GPUs.
  optimizer = keras.optimizers.RMSprop(lr=learning_rate * hvd.size())
#   optimizer = keras.optimizers.Adadelta(lr=learning_rate * hvd.size())
  
#   optimizer =RAdam(total_steps=5000, warmup_proportion=0.1, learning_rate=learning_rate*hvd.size(), min_lr=1e-5)

  # Horovod: add Horovod Distributed Optimizer.
  optimizer = hvd.DistributedOptimizer(optimizer)

  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  callbacks = [
      # Horovod: broadcast initial variable states from rank 0 to all other processes.
      # This is necessary to ensure consistent initialization of all workers when
      # training is started with random weights or restored from a checkpoint.
      hvd.callbacks.BroadcastGlobalVariablesCallback(0),
#       hvd.callbacks.MetricAverageCallback(),
#       hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
#       keras.callbacks.ReduceLROnPlateau(patience=10, verbose=1)

  ]

  # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
  if hvd.rank() == 0:
      callbacks.append(keras.callbacks.ModelCheckpoint(checkpoint_dir + 'checkpoint-{epoch}.ckpt', save_weights_only = True))

  end_time = round((time.time() - start_time),3)
  print(f"step - prep_model - {end_time}") 
  
  # Model Train
  start_time = time.time()
  
  history = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = step_size_train // hvd.size() ,
    validation_data = validation_generator,
    validation_steps = step_size_validation // hvd.size() , ## Not to oversample dataset
    epochs = epochs,
    callbacks = callbacks,
    verbose=2
    )#validation_steps=VALIDATION_STEPS,steps_per_epoch=STEPS_PER_EPOCH, I don't use them either here not only because I have a fixed sample size, and also I don't have to over sample validation set. 
  
  end_time = round((time.time() - start_time),3)
  print(f"step - train_model - {end_time}")   
    

In [23]:
reset_stdout()

In [24]:
from sparkdl import HorovodRunner
instance_type = "HOROVOD_CLUSTER"

Using TensorFlow backend.

In [25]:
np_list = [-8, 1, 8]

In [26]:
for np_setup in np_list:
  
  start_time = time.time()
  reset_stdout()
  
  checkpoint_dir = '/dbfs/ml/vgg16/train/{}/{}/'.format(np_setup, time.time())
  os.makedirs(checkpoint_dir)
  
  filename = f"benchmark|{env}|{dataset}|{model_name}|{train_shape}|{validation_shape}|{instance_type}|{driver_type}|{worker_type}|{num_workers}|{np_setup}|{repeat}|{epochs}|{learning_rate}|{batch_size}.log"
  print(output_dir+filename)
  redirect_stdout(output_dir+filename)

  for i in range(repeat):
    print(f"{instance_type} - REPEAT {i+1}")
    hr = HorovodRunner(np = np_setup)
    hr.run(train_hvd,  learning_rate=learning_rate)

  reset_stdout()
  end_time = round((time.time() - start_time),3)
  print(f"np{np_setup} - finshed in {end_time}") 
  
  print("Saving to s3....")
  save_horovod_model()
  move_log_to_s3()
  print("Saving to s3 finshed!")

--------------------------------------------------------------------------- 
 RuntimeError Traceback (most recent call last)
 <command-11377> in <module> 
 14 print ( f"{instance_type} - REPEAT {i+1}" ) 
 15 hr = HorovodRunner ( np = np_setup ) 
 ---> 16 hr . run ( train_hvd , learning_rate = learning_rate ) 
 17 
 18 reset_stdout ( ) 

 /databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.5.0-db5-spark2.4_shaded.jar/sparkdl/utils/instrumentation.py in instrumented (func, self, args, kwargs) 
 24 self.__class__.__name__))
 25 try : 
 ---> 26 return_val = func ( * args , ** kwargs ) 
 27 except Exception : 
 28 error_string = traceback . format_exc ( ) 

 /databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.5.0-db5-spark2.4_shaded.jar/sparkdl/horovod/runner.py in run (self, main, **kwargs) 
 161 root_working_dir = os . path . join ( tempfile . gettempdir ( ) , run_id ) 
 162 self . _last_root_working_dir = root_working_dir
 --> 163 return self . _run_program ( root_working_dir , pickled_func_str ) 
 164 
 165 @ staticmethod

 /databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.5.0-db5-spark2.4_shaded.jar/sparkdl/horovod/runner.py in _run_program (self, root_working_dir, pickled_func_str) 
 404 cmd = HorovodRunner._get_mpi_command(
 405 hosts, args, extra_mpi_args=[], private_key_path=ssh_session.private_key_path)
 --> 406 HorovodRunner . _run_command ( cmd ) 
 407 local_bytes = HorovodRunner . _get_main_result_bytes ( root_working_dir ) 
 408 result = HorovodRunner . _parse_result_bytes ( local_bytes ) 

 /databricks/jars/third_party--mlruntime--spark-deep-learning--com.databricks__spark-deep-learning_2.11__1.5.0-db5-spark2.4_shaded.jar/sparkdl/horovod/runner.py in _run_command (args, _prctl, log_streaming_client) 
 246 check the executor logs for Task 0 in the Spark UI . 
 247 """ % last_n_msg) +
 --> 248 "\n%s\n" % "".join(tail))
 249 
 250 @ staticmethod

 RuntimeError : Command ['mpirun', '--allow-run-as-root', '-np', '8', '-H', 'localhost,localhost,localhost,localhost,localhost,localhost,localhost,localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_98742d488d6e/id_rsa', 'bash', '/tmp/HorovodRunner_98742d488d6e/launch.sh'] failed with return code 1.
The last 100 lines of the task output are included below. If you're having trouble accessing the
full output, check the executor logs for Task 0 in the Spark UI.
[1,0]<stderr>: File "/databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py", line 1881, in _maybe_build
[1,0]<stderr>: self.build(input_shapes)
[1,0]<stderr>: File "/databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py", line 1017, in build
[1,0]<stderr>: trainable=True)
[1,0]<stderr>: File "/databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py", line 384, in add_weight
[1,0]<stderr>: aggregation=aggregation)
[1,0]<stderr>: File "/databricks/python/lib/python3.7/site-packages/tensorflow/python/training/tracking/base.py", line 663, in _add_variable_with_custom_getter
[1,0]<stderr>: **kwargs_for_getter)
[1,0]<stderr>: File "/databricks/python/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer_utils.py", line 155, in make_variable
[1,0]<stderr>: shape=variable_shape if variable_shape.rank else None)
[1,0]<stderr>: File "/databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/variables.py", line 259, in __call__
[1,0]<stderr>: return cls._variable_v1_call(*args, **kwargs)
[1,0]<stderr>: File "/databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/variables.py", line 220, in _variable_v1_call
[1,0]<stderr>: shape=shape)
[1,0]<s

In [28]:
for np_setup in np_list:
  
  start_time = time.time()
  reset_stdout()
  
  checkpoint_dir = '/dbfs/ml/vgg16/train/{}/{}/'.format(np_setup, time.time())
  os.makedirs(checkpoint_dir)
  
  filename = f"benchmark|{env}|{dataset}|{model_name}|{train_shape}|{validation_shape}|{instance_type}|{driver_type}|{worker_type}|{num_workers}|{np_setup}|{repeat}|{epochs}|{learning_rate}|{batch_size}.log"
  print(output_dir+filename)
  redirect_stdout(output_dir+filename)

  for i in range(repeat):
    print(f"{instance_type} - REPEAT {i+1}")
    hr = HorovodRunner(np = np_setup)
    hr.run(train_hvd,  learning_rate=learning_rate)

  reset_stdout()
  end_time = round((time.time() - start_time),3)
  print(f"np{np_setup} - finshed in {end_time}") 
  
  print("Saving to s3....")
  save_horovod_model()
  move_log_to_s3()
  print("Saving to s3 finshed!")